# Concepts in Network Theory

Networks: Network representations are ubiquitous for things like social 
networks, the world wide web, transportation and power grids. 

In biology it naturally projects to dynamics of molecular interaction inside cells 
(metabolic or protein-protein networks), neuroscience and and ecological networks. In this submodule we will 
focus on the first two. As an example of an abstract network, the image below of [Disease network](https://barabasi.com/f/320.pdf) is constructed from protein-protein interactions (two diseases are considered connected if their disease-associated genes interact, or if they share such a gene).

<img src="assets/barabasi_disease_network.png" alt="disease network" style="float:left" width="400" />


Network theory is also being increasingly used in neuroscience, coined **connectomics**. Nothing about the idea is new, as it dates back at last 100 years, but only with advances in imaging technologies have we started to gain access into what it looks like. Some neuroscientists believe that if we had perfect resolution of an individual's connectome, this would (in theory) allow full access into that person's memories, experiences, knowledge and personality.

We leave you with a few articles to appreciate network theory as a tool to study biology.

https://barabasi.com/f/320.pdf

https://www.nature.com/articles/nrg.2016.87

https://www.nature.com/articles/nrg1272

For great explanations of more or less advanced network science concepts and algorithms in Python, we recommend to check out [T.M.]()

## Network representation

Networks are most naturally visualized as a set of points (nodes) connected by lines (edges).

<img
src="assets/graph_random_circle.png"
width=300
/>

This data is typically stored as a list of edges, for the figure above we would have G = {A,B}, {B,A}, {B,C}.
Another equivalent form is the adjacency matrix, which is less intuitive, but mathematically appealing (color code: black=1; white=0).


<img
src="assets/matrix_example.png"
width=500
/>

And as matrix of numbers:

<img
src="assets/adjacency_matrix.JPG"
width=500
/>

In [ ]:
import numpy as np

<div class='alert alert-warning'>
<h4>Exercise 1.</h4>
    construct the matrix (numpy array of 1s and 0s) <b>A</b> representing the following graph. You don't need to mind the arrows.
</div>

<img
src="assets/ex0_1.png"
width=200
style="float: left"
/>

**Hint:** we write the connection from row to column (i.e. the link from 1st node to 3rd should be at the 1st row and 3rd column). Note this convention is not always consistent.

**NB.** It doesn't really matter what is the *1st* node. You can start anywhere. 

In [ ]:
# Ex1


In [ ]:
# %load solutions/ex0_1.py

---

## NetworkX

`NetworkX` is a python library specialized for working with graphs. They provide a class `networkx.Graph` which stores a network as a list of edges. We can convert freely between the different representations using `networkx`.

In [ ]:
import networkx as nx
G = nx.from_numpy_array(A)  #create network using numpy array A

In [ ]:
nx.draw(G)  # draw the network

## Directedness and weightedness

We have two questions of network properties to consider when making a model:
- is the edges directed or undirected? 
- are the edges continuous or binary?

For instance, the WWW is directed, but a social acquaintance does not follow a particular direction. We can use the `nx.DiGraph` class to force it to create a directed graph.

In [ ]:
G_dir = nx.from_numpy_array(A, create_using=nx.DiGraph) # create directed graph using numpy array A
nx.draw(G_dir)

And we can just as easily transform it back into a numpy array:

In [ ]:
M = nx.to_numpy_array(G_dir)
M == A #they are the same as expected

## Graph metrics
Graph metrics are numbers that quanitfy certain properties of a network (global metrics) or about specific nodes in the network (local metrics). Because it is hard to infer things about a network simply by looking at it, these numbers capture the essence of a network, and lets us test specific hypotheses about network structure.

<img src="assets/rubinov_sporns_2010_neuroimage.png"
     alt="graph metrics"
     width=1000
     style="float: left; margin-right: 10px;" />

#### A bunch of other graph metrics exist that quantify importance of a node within a network.
These are collectively termed "centrality", but all quantify slightly different things.

- **degree centrality**
   - the number of edges the node has, or the proportion of maximum amount of edges. The higher degree, the more central the node is.
- **betweenness centrality**
   - Between each pair of nodes there is a shortest path between them. Betweennees centrality represents how many of these paths go through the node.
- closeness centrality

#### Other metrics quantify overall network structure
- **clustering coefficient** 
   - a measure of the degree to which nodes in a graph tend to cluster together
- density
- efficiency
- small-worldness
- modularity



The various metrics are tempting to use, but it is crucial to interpret them according to the network you happen to study.

In [ ]:
from matplotlib import pyplot as plt

### Visualizing bottlenecks (betweenness centrality)


In [ ]:
# let's use the same graph
G = nx.from_numpy_array(A)

Try to figure out what the numbers in these plots actually mean:

In [ ]:
# try to find hub nodes
btw = nx.betweenness_centrality(G)  

# visualize bottlenecks
pos = nx.layout.spring_layout(G, seed=1)
nx.draw(G, pos=pos, labels=btw, node_color='lightblue', font_size=18, font_color='k')
plt.title('Bottleneck centrality')

In [ ]:
deg = nx.centrality.degree_centrality(G)
nx.draw(G, pos=pos, labels=deg, node_color='lightblue', font_size=18, font_color='k')
plt.title('Degree centrality')

Notice how the bottom node has a high degree centrality, but low bottleneck centrality. 

In [ ]:
cc = nx.clustering(G)
nx.draw(G, pos=pos, labels=cc, node_color='lightblue', font_size=18, font_color='k')
plt.title('Clustering Coefficient')

# A more advanced example of network theory in action
Consider the world wide web (WWW), one of the more familiar networks of everyday life. When performing a search with your search engine of choice, it has to sort the results based on some kind of importance metric. The old method of doing this was to base it upon page *content*, and rate it accordingly. This method yielded very poor results. An ingenious milestone was, paradoxically, to dismiss the site content, and *only* look at its **topological** position in the network.

### PageRank
We use the seemingly circular argument: a page (node) is important if other important pages point (link) to it. We start with an initial distribution of "importance" between the nodes, then at each iteration the node redistributes its own importance determined by their outward links (imagine a surfer who at each time step has a certain probability of moving to a new page). This process is allowed to repeat, and will at some point reach steady state. The below image has already reached steady state.


<img src="assets/PageRanks-Example_345Kai_no(c).jpg"
     alt="graph metrics"
     width=500
     style="float: left; margin-right: 10px;" />
(image credit to 345Kai wikipedia)

We will go on to reproduce the above figure.

In [ ]:
np.set_printoptions(2)

In [ ]:
# define the edges. A is set to link to all other pages
#(a mathematical necessity because it has no outward links)

M = np.array([
       [1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0.],
       [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.],
       [1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

<div class='alert alert-warning'> <h4>Exercise 2. </h4> Normalize the edges of <b>M</b> such that the sum of the columns equal to 1.

**(a)** First compute the column sums (yielding a vector),**(b)** then divide each column by that vector (broadcasting). **(c)** Verify that the columns sum to one.
    </div>

In [ ]:
# a)


In [ ]:
# %load solutions/ex0_2a.py

In [ ]:
# b)


In [ ]:
# load solutions/ex0_2b.py

In [ ]:
# c)


In [ ]:
# %load solutions/ex0_2c.py

### The pagerank algorithm

In [ ]:
# don't worry about this code
# from https://en.wikipedia.org/wiki/PageRank#Simplified_algorithm
import numpy as np

def pagerank(M, num_iterations: int = 100, d: float = 0.85):
    """PageRank: The trillion dollar algorithm.

    Parameters
    ----------
    M : numpy array
        adjacency matrix where M_i,j represents the link from 'j' to 'i', such that for all 'j'
        sum(i, M_i,j) = 1
    num_iterations : int, optional
        number of iterations, by default 100
    d : float, optional
        damping factor, by default 0.85

    Returns
    -------
    numpy array
        a vector of ranks such that v_i is the i-th rank from [0, 1],
        v sums to 1

    """
    N = M.shape[1]
    v = np.random.rand(N, 1)
    v = v / np.linalg.norm(v, 1)
    M_hat = (d * M + (1 - d) / N)
    for i in range(num_iterations):
        v = M_hat @ v
    return v

<div class='alert alert-warning'> <h4>Exercise 3:</h4> 
Compute the pagerank (call it `pr`) and run visualization belowe to confirm it matches with the figure shown above and at the end of notebook (within a reasonable margin of error).
    </div>

In [ ]:
# Ex3


In [ ]:
# %load solutions/ex0_3.py

## Visualize that the results check out

In [ ]:
# don't worry about this code

names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K']
colors = ['lightblue', 'yellow', 'orange', 'purple', 'pink', 'purple', 'cyan','cyan','cyan','cyan','cyan']

G = nx.from_numpy_array(M.T, create_using=nx.DiGraph)
nx.relabel_nodes(G, lambda k: dict(zip(range(11), names))[k], copy=False)

# remove edges
rem = [edge for edge in G.edges if edge[0]=='A']
G.remove_edges_from(rem)
pos = nx.layout.circular_layout(G)

pr = dict(zip(names, pr.flatten()))

for k,v in pr.items():
    pr[k] = round(v*100, 1)
    
node_size = np.array(list(pr.values()))*150

nx.draw(G, pos=pos, node_color=colors, with_labels=True, node_size=node_size)
for k,v in pos.items(): v[1]-=0.1
nx.draw_networkx_labels(G, labels=pr, pos=pos)
plt.show()

<img src="assets/PageRanks-Example_345Kai_no(c).jpg"
     alt="graph metrics"
     width=500
     style="float: left; margin-right: 10px;" />

This example was simply for illustration, we could just as easily have used networkX's own implementation of pagerank.